In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace. I _still_ run up against resource limits and have designed the code to contend with them.

The code affords some options to keep things efficient. Throughout the workflow you can specify `compress=True`, which tells the code to compress its output files. Downstream scripts will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for  many of the scripts; this will delete the source files for a step once that step is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

`[Total runtime: ~50 minutes]`

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors).

`[Runtime: 0:00:42.417839]`

In [3]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    file_range=(0, 24),
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False
)

Start Time:                2025-01-09 08:51:21.155161

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/eng-1-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download
File index range:          0 to 24
File URLs available:       24
File URLs to use:          24
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00000-of-00024.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00023-of-00024.gz
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  False


End Time:                  2025-01-09 08:52:03.573000
Total runtime:             0:00:42.417839



### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

`[Runtime: 0:02:28.980029]`

In [4]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-09 08:52:35.420040

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download/1-00000-of-00024.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download/1-00023-of-00024.txt.lz4
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Delete input directory:    True

Converting      |██████████████████████████████████████████████████| 100.0% 24          /24         

End Time:                  2025-01-09 08:55:04.400069
Total runtime:             0:02:28.980

### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

`[Runtime: 0:01:07.284543]`

In [5]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-09 08:57:23.795844

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert/1-00000-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert/1-00023-of-00024.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Delete input directory:    True

Lowercasing     |██████████████████████████████████████████████████| 100.0% 24          /24         

End Time:                  2025-01-09 08:58:31.080387
Total runtime:             0:01:07.284543



### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

`[Runtime: 0:01:22.585907]`

In [6]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=False,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-09 08:58:47.841608

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase/1-00000-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase/1-00023-of-00024.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     False
Overwrite existing files:  False
Delete input directory:    True

Lemmatizing     |██████████████████████████████████████████████████| 100.0% 24          /24         

End Time:                  2025-01-09 09:00:10.427515
Total runtime:             0:01:22.585907



### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

`[Runtime: 0:01:14.249064]`

In [7]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    compress=True,
    overwrite=False,
    delete_input=True
)

Start Time:                   2025-01-09 09:16:23.635937

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter
File index range:             0 to 23
Files available:              24
Files to use:                 24
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize/1-00000-of-00024.jsonl
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize/1-00023-of-00024.jsonl
Ngram size:                   1
Number of workers:            48
Compress output files:        True
Overwrite existing files:     False
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          True
Drop ngrams under:            1

### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

`[Runtime: 0:21:37.356843]`

In [2]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=10,
    sort_key='ngram',
    compress=True,
    overwrite=True,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-01-09 10:04:27.987236

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4
Files available:           18
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter/1-00006-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter/1-00023-of-00024.jsonl.lz4
Files to use:              18
Ngram size:                1
Number of workers:         10
Compress output files:     True
Overwrite existing files:  True
Sort key:                  ngram
Sort order:                ascending
Heap-merge start iter:  

### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [3]:
check_file_sorted(
    input_file='/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4',
    field="ngram",
    sort_order="ascending"
)

Lines: 25127726line [05:10, 80841.55line/s] 


True

### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime: 0:12:33.459340]`

In [2]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    lines_per_chunk=1000000,
    compress=True,
    overwrite=False
)

Start Time:                2025-01-09 10:48:20.846337

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Temporary file:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/temp_chunks
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/temp_chunks
Ngram size:                1
Compress output files:     True
Overwrite existing files:  False



Process ForkPoolWorker-34:unks: 9320675 chunks [06:22, 24391.44 chunks/s]
Process ForkPoolWorker-29:
Traceback (most recent call last):
  File "/ext3/miniforge3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/ext3/miniforge3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/ext3/miniforge3/lib/python3.12/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/scratch/edk202/hist_w2v/src/ngram_tools/consolidate_ngrams.py", line 204, in consolidate_chunk
    with output_handler.open() as outfile:
         ^^^^^^^^^^^^^^^^^^^^^
  File "/ext3/miniforge3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/ext3/miniforge3/lib/python3.12/site-packages/lz4/frame/__init__.py", line 567, in close
    self._fp.close()
  File "/ext3/mini

KeyboardInterrupt: 

### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-consolidated-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-consolidated-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

`[Runtime: 0:12:02.432830]`

In [18]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    vocab_n=80000
)

Start Time:                2025-01-04 15:08:54.835535

Indexing Info
Consolidated file:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-consolidated.jsonl.lz4
Indexed file:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-indexed.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False
Vocab size (top N):        80000

Chunking        |██████████████████████████████████████████████████| 100.0% 13499384    /13499384   
Sorting         |██████████████████████████████████████████████████| 100.0% 135         /135        
Merging         |██████████████████████████████████████████████████| 100.0% 13499384    /13499384   
Indexing        |██████████████████████████████████████████████████| 100.0% 13499384    /13499384   

Indexed 13499384 lines.
Final indexed file: 1gram-consolidated-indexed.jsonl.lz4
Created vocab_list_match and vocab_lis

### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [21]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '1gram_files/6corpus/1gram-consolidated-indexed.jsonl.lz4'
    ),
    start_line=1100050,
    end_line=1100050,
    parse_json=True
)

Line 1100050: {'ngram': 'zpinch', 'freq_tot': 3384, 'doc_tot': 2350, 'freq': {'1960': 7, '1962': 5, '1964': 1, '1967': 10, '1968': 20, '1969': 52, '1970': 14, '1971': 48, '1972': 13, '1973': 73, '1976': 43, '1977': 60, '1978': 158, '1979': 74, '1981': 59, '1982': 120, '1983': 113, '1984': 106, '1985': 134, '1986': 237, '1987': 192, '1988': 181, '1989': 285, '1990': 111, '1991': 67, '1992': 60, '1993': 101, '1994': 153, '1995': 92, '1996': 43, '1997': 53, '1998': 77, '1999': 84, '2000': 20, '2001': 48, '2002': 50, '2003': 33, '2004': 48, '2005': 17, '2006': 32, '2007': 21, '2008': 10, '2010': 3, '2011': 7, '2012': 35, '2013': 11, '2014': 4, '2018': 7, '2019': 8, '1961': 2, '1965': 19, '1904': 1, '1951': 1, '1963': 9, '1966': 7, '1974': 42, '1975': 28, '1980': 61, '2009': 2, '2015': 4, '2016': 2, '2017': 5, '1919': 1}, 'doc': {'1960': 7, '1962': 5, '1964': 1, '1967': 9, '1968': 17, '1969': 44, '1970': 13, '1971': 33, '1972': 13, '1973': 52, '1976': 42, '1977': 42, '1978': 91, '1979': 65,